In [1]:
import tacco as tc
import anndata
import pandas as pd
import numpy as np
import scanpy as sc

In [2]:
z_stack = 3

In [3]:
andata_sp = anndata.read(f'/media/gambino/students_workdir/ilias/data/images/tile40/transcripts/tile_40_gt_z{z_stack}_expression_matrix.h5ad')
andata_sc = anndata.read_text('/media/gambino/students_workdir/ilias/data/colorectal_cancer_sc/GSE132465_GEO_processed_CRC_10X/GSE132465_GEO_processed_CRC_10X_raw_UMI_count_matrix.txt.gz').transpose()

In [4]:
cell_annotation_sc = pd.read_table('/media/gambino/students_workdir/ilias/data/colorectal_cancer_sc/GSE132465_GEO_processed_CRC_10X/GSE132465_GEO_processed_CRC_10X_cell_annotation.txt.gz',index_col=0)
andata_sc.obs = cell_annotation_sc

In [5]:
andata_sp

AnnData object with n_obs × n_vars = 12244 × 500
    obs: 'mask_label', 'bbox', 'centroid-0', 'centroid-1', 'axis_major_length', 'axis_minor_length', 'area'
    uns: 'image', 'img_files', 'masks', 'transcript_coordinates'

In [6]:
sc.pp.filter_cells(andata_sp, min_genes=30)
sc.pp.filter_genes(andata_sp, min_cells=3)

In [7]:
andata_sp

AnnData object with n_obs × n_vars = 12135 × 500
    obs: 'mask_label', 'bbox', 'centroid-0', 'centroid-1', 'axis_major_length', 'axis_minor_length', 'area', 'n_genes'
    var: 'n_cells'
    uns: 'image', 'img_files', 'masks', 'transcript_coordinates'

In [8]:
if np.any(andata_sp.var_names.str.startswith('MT-')):
    andata_sp.var['mt'] = andata_sp.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
    sc.pp.calculate_qc_metrics(andata_sp, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
else:
    sc.pp.calculate_qc_metrics(andata_sp, percent_top=None, log1p=False, inplace=True)

## TACCO

In [11]:
annotations_tacco_cell_type = tc.tl.annotate(andata_sp, andata_sc, annotation_key='Cell_type')
annotations_tacco_class = tc.tl.annotate(andata_sp, andata_sc, annotation_key='Class')

Starting preprocessing
Finished preprocessing in 1.25 seconds.
Starting annotation of data with shape (12135, 497) and a reference of shape (63688, 497) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Cell_type, normalize_to=adata
   +- bisection boost: bisections=4, bisection_divisor=3
      +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  2.669033144591657 15.66134505752349
bisection run on 1


/media/gambino/students_workdir/ilias/miniconda3/envs/TACCO_env/lib/python3.10/site-packages/tacco/tools/_annotate.py:177: FutureWarning: X.dtype being converted to np.float32 from uint32. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(adata.X.copy(), obs=adata.obs[[]], var=adata.var[[]])


bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 1.62 seconds.
Starting preprocessing
Annotation profiles were not found in `reference.varm["Class"]`. Constructing reference profiles with `tacco.preprocessing.construct_reference_profiles` and default arguments...
Finished preprocessing in 1.48 seconds.
Starting annotation of data with shape (12135, 497) and a reference of shape (63688, 497) using the following wrapped method:
+- platform normalization: platform_iterations=0, gene_keys=Class, normalize_to=adata
   +- bisection boost: bisections=4, bisection_divisor=3
      +- core: method=OT annotation_prior=None
mean,std( rescaling(gene) )  2.669033144591657 15.66134505752349
bisection run on 1


/media/gambino/students_workdir/ilias/miniconda3/envs/TACCO_env/lib/python3.10/site-packages/tacco/tools/_annotate.py:177: FutureWarning: X.dtype being converted to np.float32 from uint32. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(adata.X.copy(), obs=adata.obs[[]], var=adata.var[[]])


bisection run on 0.6666666666666667
bisection run on 0.4444444444444444
bisection run on 0.2962962962962963
bisection run on 0.19753086419753085
bisection run on 0.09876543209876543
Finished annotation in 1.63 seconds.


In [13]:
andata_sp_tacco = andata_sp.copy()
andata_sp_tacco.obs['pred_cell_type'] = annotations_tacco_cell_type.idxmax(axis=1)
andata_sp_tacco.obs['pred_class'] = annotations_tacco_class.idxmax(axis=1)

In [14]:
andata_sp_tacco.obs

,mask_label,bbox,centroid-0,centroid-1,axis_major_length,axis_minor_length,area,n_genes,n_genes_by_counts,total_counts,pred_cell_type,pred_class
cell_id,,,,,,,,,,,,
7596_0_2010_93_2113,7596,"0,2010,93,2113",45.507658,2057.688285,109.680729,85.044677,7247.0,165,165,546,Stromal cells,Tumor
7597_37_3330_93_3396,7597,"37,3330,93,3396",61.828131,3364.045538,68.568696,51.853015,2723.0,99,99,258,Epithelial cells,Tumor
7598_37_3450_167_3552,7598,"37,3450,167,3552",108.106342,3506.736172,131.445293,91.276359,8877.0,198,198,736,Epithelial cells,Tumor
7599_111_4659_167_4751,7599,"111,4659,167,4751",137.323662,4699.303434,93.839303,52.468402,3757.0,155,155,502,Myeloids,Tumor
7600_157_4871_214_4937,7600,"157,4871,214,4937",183.948817,4900.310585,69.310639,50.791298,2579.0,55,55,126,Epithelial cells,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...
705140_8122_6052_8197_6127,705140,"8122,6052,8197,6127",8155.671742,6087.641292,77.904241,71.367987,4335.0,177,177,563,Myeloids,Normal
705141_8131_4410_8188_4448,705141,"8131,4410,8188,4448",8159.277710,4430.278928,60.143913,35.322795,1642.0,48,48,93,Epithelial cells,Tumor
705142_8150_4972_8215_5038,705142,"8150,4972,8215,5038",8179.948357,5002.281690,75.216194,53.344501,2982.0,99,99,343,Epithelial cells,Tumor


In [15]:
andata_sp_tacco.write(f'/media/gambino/students_workdir/ilias/data/images/tile40/transcripts/tile_40_gt_z{z_stack}_expression_matrix_dc_tacco.h5ad')

## TANGRAM

In [9]:
annotations_tangram_cell_type = tc.tl.annotate_tangram(andata_sp, andata_sc, annotation_key='Cell_type', cluster_mode = False,
                       conda_env='/media/gambino/students_workdir/ilias/miniconda3/envs/tangram/', device='cpu')
annotations_tangram_class = tc.tl.annotate_tangram(andata_sp, andata_sc, annotation_key='Class', cluster_mode = False,
                       conda_env='/media/gambino/students_workdir/ilias/miniconda3/envs/tangram/', device='cpu')

/media/gambino/students_workdir/ilias/miniconda3/envs/TACCO_env/lib/python3.10/site-packages/tacco/utils/_utils.py:902: FutureWarning: X.dtype being converted to np.float32 from uint32. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(adata.X, var=adata.var, obs=adata.obs)


/media/gambino/students_workdir/ilias/miniconda3/envs/tangram/bin/python
Score: 0.344, KL reg: 0.222
Score: 0.811, KL reg: 0.011
Score: 0.843, KL reg: 0.006
Score: 0.853, KL reg: 0.004
Score: 0.858, KL reg: 0.004
Score: 0.860, KL reg: 0.004
Score: 0.861, KL reg: 0.003
Score: 0.862, KL reg: 0.003
Score: 0.863, KL reg: 0.003
Score: 0.863, KL reg: 0.003

INFO:root:497 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:497 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 497 genes and rna_count_based density_prior in cells mode...
INFO:root:Printing scores every 100 epochs.
INFO:root:Saving results

/media/gambino/students_workdir/ilias/miniconda3/envs/TACCO_env/lib/python3.10/site-packages/tacco/utils/_utils.py:902: FutureWarning: X.dtype being converted to np.float32 from uint32. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = ad.AnnData(adata.X, var=adata.var, obs=adata.obs)


In [ ]:
andata_sp_tangram = andata_sp.copy()
andata_sp_tangram.obs['pred_cell_type'] = annotations_tangram_cell_type.idxmax(axis=1)
andata_sp_tangram.obs['pred_class'] = annotations_tangram_class.idxmax(axis=1)

In [ ]:
andata_sp_tangram.obs

,mask_label,bbox,centroid-0,centroid-1,axis_major_length,axis_minor_length,area,n_genes,n_genes_by_counts,total_counts,pred_cell_type
cell_id,,,,,,,,,,,
7596_0_2010_93_2113,7596,"0,2010,93,2113",45.507658,2057.688285,109.680729,85.044677,7247.0,165,165,546,Stromal cells
7597_37_3330_93_3396,7597,"37,3330,93,3396",61.828131,3364.045538,68.568696,51.853015,2723.0,99,99,258,Stromal cells
7598_37_3450_167_3552,7598,"37,3450,167,3552",108.106342,3506.736172,131.445293,91.276359,8877.0,198,198,736,Epithelial cells
7599_111_4659_167_4751,7599,"111,4659,167,4751",137.323662,4699.303434,93.839303,52.468402,3757.0,155,155,502,Myeloids
7600_157_4871_214_4937,7600,"157,4871,214,4937",183.948817,4900.310585,69.310639,50.791298,2579.0,55,55,126,B cells
...,...,...,...,...,...,...,...,...,...,...,...
705140_8122_6052_8197_6127,705140,"8122,6052,8197,6127",8155.671742,6087.641292,77.904241,71.367987,4335.0,177,177,563,Myeloids
705141_8131_4410_8188_4448,705141,"8131,4410,8188,4448",8159.277710,4430.278928,60.143913,35.322795,1642.0,48,48,93,T cells
705142_8150_4972_8215_5038,705142,"8150,4972,8215,5038",8179.948357,5002.281690,75.216194,53.344501,2982.0,99,99,343,Epithelial cells


In [ ]:
andata_sp_tangram.write(f'/media/gambino/students_workdir/ilias/data/images/tile40/transcripts/tile_40_gt_z{z_stack}_expression_matrix_dc_tangram.h5ad')